<a href="https://colab.research.google.com/github/sufyanAshraf/AI_Agents/blob/main/groq_langchain_Faiss_lama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_groq

In [ ]:
!pip install langchain_community

In [ ]:
from groq import Groq

Fill 3 keys

In [ ]:
langchain_key ="" # 1 key

In [ ]:
api_key = "gsk_*" # 2 key

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')
userdata.get('LANGCHAIN_API_KEY')

'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'

In [ ]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores.cassandra import Cassandra
from langchain_community.vectorstores import FAISS

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs=text_splitter.split_documents(text_documents)
# docs[:5]

In [ ]:
!pip install sentence-transformers

In [ ]:
from huggingface_hub import login

login("") # 3 key


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


<ipython-input-15-ab9b809b138b>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 30.9 MB/s eta 0:00:00


In [ ]:
db = FAISS.from_documents(docs[:30], embeddings)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>

Question: {input}""")

In [ ]:
llm=ChatGroq(groq_api_key=api_key,
         model_name="mixtral-8x7b-32768")

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x79a9772ee790>, search_kwargs={})

In [ ]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response=retrieval_chain.invoke({"input":"Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique"})

In [ ]:
response['answer']

'for enhancing model performance on complex tasks. It works by instructing the language model to "think step by step" to decompose hard tasks into smaller and simpler steps. This process, known as task decomposition, can be done in three ways:\n\n1. By using a large language model (LLM) with simple prompting, such as "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?"\n2. By using task-specific instructions, such as "Write a story outline." for writing a novel\n3. With human inputs\n\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thoughts. The search process can be BFS or DFS, with each state evaluated by a classifier or majority vote.\n\nReAct (Yao et al. 2023) integrates reasoning and acting within LLM by extending the action space to include task-specific discrete actions, enabling LLM to interact with the environment, and the language space, prompting LLM to generate reasoning 

# 1 simple e.g

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are assistant"),
    ("user", "question {input}")
])

In [ ]:
x = "What is the capital of France?"

In [ ]:
out = StrOutputParser()
chain = prompt | llm | out

In [ ]:
print(chain.invoke({x}))

The capital of France is Paris. Paris is one of the most populous cities in Europe, and is renowned as a center for art, fashion, gastronomy, and culture. The city is home to numerous iconic landmarks, including the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum.


# 2 langserve

In [ ]:
pip install langserve

In [ ]:
pip install fastapi

In [ ]:
pip install uvicorn

In [ ]:
from fastapi import FastAPI

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langserve import add_routes
import uvicorn

In [ ]:

llama=ChatGroq(groq_api_key=api_key,
         model_name="llama3-8b-8192")

In [ ]:
llm=ChatGroq(groq_api_key=api_key,
         model_name="mixtral-8x7b-32768")

In [ ]:
prompt1 = ChatPromptTemplate.from_template("""
write me a poem about {topic}
""")

prompt2 = ChatPromptTemplate.from_template("""
write me a 50 words essay about {topic}
""")

In [ ]:
app = FastAPI()

In [ ]:
add_routes(app,prompt2|llama,path="/essay")

In [ ]:
add_routes(app,prompt1|llm ,path="/poem")


# 3 Tools

In [ ]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=2679de43d5fa74c4402185604a2501c6660b224bc8318dcd204e2d80fd19783a
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [ ]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=70c887e724447a8b3b12ddf4eae22711f601d00b0257ca1ac3afbcaf5f208d0e
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


### tool 1 wikipedia

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
wiki.name

'wikipedia'

### tool 2 vectordb


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x79a80c743bd0>, search_kwargs={})

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [ ]:

retriever_tool.name

'langsmith_search'

### tool 3 arxiv

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

### next

In [ ]:
tools=[wiki,arxiv,retriever_tool]

In [ ]:
llm=ChatGroq(groq_api_key=api_key,
         model_name="mixtral-8x7b-32768")

In [ ]:
pip install langchainhub

In [ ]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})

# if you use openAI it will work

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
